In [4]:
!git clone https://github.com/jan1na/Neural-Cellular-Automata.git

%cd Neural-Cellular-Automata

Cloning into 'Neural-Cellular-Automata'...
remote: Enumerating objects: 169, done.
remote: Counting objects: 100% (169/169), done.
remote: Compressing objects: 100% (123/123), done.
remote: Total 169 (delta 102), reused 78 (delta 42), pack-reused 0 (from 0)
Receiving objects: 100% (169/169), 766.68 KiB | 11.62 MiB/s, done.
Resolving deltas: 100% (102/102), done.
/content/Neural-Cellular-Automata/Neural-Cellular-Automata


In [5]:
!pip install -U gdown

file_ids = ['1W7y-7ebqUCFW2ZPEsafABw_OPnXRr6cj',
            '1L99QItxhTDj3dS1dd8ubmxka4FxXb2HG',
            '1PqrOpn4NJF3xp4Jw2KtRRVD_20Feucsc',
            '1DtZJ5eD2PV4sO51MMQy9CVTZxgBKj_Gx']

file_names = ['pathmnist.npz', 'pathmnist_64.npz', 'pathmnist_128.npz', 'pathmnist_224.npz']

for file_id, file_name in zip(file_ids, file_names):
    !gdown --id {file_id} --output {file_name}

/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1W7y-7ebqUCFW2ZPEsafABw_OPnXRr6cj
From (redirected): https://drive.google.com/uc?id=1W7y-7ebqUCFW2ZPEsafABw_OPnXRr6cj&confirm=t&uuid=ea911095-04ae-4bb8-920c-5d1f8414f968
To: /content/Neural-Cellular-Automata/Neural-Cellular-Automata/pathmnist.npz
100% 206M/206M [00:01<00:00, 143MB/s]
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1L99QItxhTDj3dS1dd8ubmxka4FxXb2HG
From (redirected): https://drive.google.com/uc?id=1L99QItxhTDj3dS1dd8ubmxka4FxXb2HG&confirm=t&uuid=e4843ff9-fba3-

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
!pip install -q medmnist scikit-learn torchmetrics

import torch
from torch.utils.data import DataLoader
from torchvision import transforms
from torchmetrics.classification import MulticlassCalibrationError
from medmnist import PathMNIST
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, balanced_accuracy_score, mean_absolute_error, brier_score_loss
from sklearn.preprocessing import label_binarize
import seaborn as sns
from models import NCA, CNNBaseline
import torch.nn.functional as F
import numpy as np



device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.5/963.5 kB 26.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 34.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 72.1 MB/s eta 0:00:00
U

In [8]:
from google.colab import drive
import os
import shutil

drive_folder = "/content/Neural-Cellular-Automata/"
cache_dir = os.path.expanduser("~/.medmnist")
os.makedirs(cache_dir, exist_ok=True)

resolutions = ["", "_64", "_128", "_224"]
for res in resolutions:
    filename = f"pathmnist{res}.npz"
    src = os.path.join(drive_folder, filename)
    dst = os.path.join(cache_dir, filename)

    if os.path.exists(src):
        shutil.copyfile(src, dst)
        print(f"Copied {filename} to cache.")
    else:
        print(f"File not found in Drive: {filename}")

Copied pathmnist.npz to cache.
Copied pathmnist_64.npz to cache.
Copied pathmnist_128.npz to cache.
Copied pathmnist_224.npz to cache.


In [9]:
nca = NCA().to(device)
nca.load_state_dict(torch.load("/content/drive/MyDrive/NCA/best_nca_pathmnist.pth"))
nca.eval()

cnn = CNNBaseline().to(device)
cnn.load_state_dict(torch.load("/content/drive/MyDrive/NCA/best_cnn_pathmnist.pth"))
cnn.eval()

CNNBaseline(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
  )
  (pool): AdaptiveAvgPool2d(output_size=(1, 1))
  (fc): Linear(in_features=128, out_features=9, bias=True)
)

In [10]:
def get_loader(size, batch_size=64):
    transform = transforms.Compose([transforms.ToTensor()])
    dataset = PathMNIST(split="test", size=size, download=False, transform=transform)
    return DataLoader(dataset, batch_size=batch_size, shuffle=False)

In [11]:
@torch.no_grad()
def evaluate(model, loader, name="Model", size=28, save_dir="/content/drive/MyDrive/NCA/results", is_NCA=False):
    all_preds, all_labels, all_logits = [], [], []

    for x, y in loader:
        x, y = x.to(device), y.squeeze()
        if is_NCA:
            out, rgb_steps = model(x, True)
        else:
            out = model(x)

        all_logits.append(out.cpu())
        pred = out.argmax(dim=1).cpu().numpy()
        label = y.numpy()
        all_preds.extend(pred)
        all_labels.extend(label)

    # Converting for scores
    logits = torch.cat(all_logits)  # (N, C)
    probs = F.softmax(logits, dim=1).numpy()  # (N, C)
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    # Metrics
    cm = confusion_matrix(all_labels, all_preds)
    overall_acc = accuracy_score(all_labels, all_preds)
    bal_acc = balanced_accuracy_score(all_labels, all_preds)
    mae = mean_absolute_error(all_labels, all_preds)
    report = classification_report(all_labels, all_preds, digits=4)


    ### Uncertainty Quantification (NQM)

    # Brier Score
    y_true_bin = label_binarize(all_labels, classes=list(range(probs.shape[1])))
    brier = np.mean(np.sum((probs - y_true_bin) ** 2, axis=1))

    # NLL (Cross-Entropy)
    all_labels_tensor = torch.tensor(all_labels)
    nll = F.cross_entropy(logits, all_labels_tensor, reduction='mean').item()

    # Entroyp
    entropy = -np.sum(probs * np.log(probs + 1e-12), axis=1)
    mean_entropy = np.mean(entropy)

    # ECE
    probs_tensor = torch.from_numpy(probs).float()
    labels_tensor = torch.from_numpy(all_labels).long()

    ece_metric = MulticlassCalibrationError(num_classes=probs.shape[1], n_bins=15, norm='l1')
    ece = ece_metric(probs_tensor, labels_tensor).item()

    # Logging
    print(f"\n{name} @ {size}x{size}")
    print(f"Overall Accuracy: {overall_acc:.4f}")
    print("Balanced Accuracy:", f"{bal_acc:.4f}")
    print("Mean Absolute Error (MAE):", f"{mae:.4f}")
    print("")
    print("Uncertainty Quantification:")
    print(f"Brier Score: {brier:.4f}")
    print(f"NLL (Cross-Entropy)  : {nll:.4f}")
    print(f"Mean Predictive Entropy: {mean_entropy:.4f}")
    print(f"Expected Calibration Error (ECE): {ece:.4f}")

    print(report)

    # Save confusion matrix
    os.makedirs(save_dir, exist_ok=True)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f"Confusion Matrix: {name} @ {size}x{size}")
    plt.xlabel("Predicted")
    plt.ylabel("True")
    fname = f"{save_dir}/cm_{name.replace(' ', '_')}_{size}x{size}.png"
    plt.savefig(fname)
    plt.close()
    print(f"Confusion matrix saved to: {fname}")

    return {
        "overall_acc": overall_acc,
        "bal_acc": bal_acc,
        "mae": mae,
        "brier": brier,
        "nll": nll,
        "entropy": mean_entropy,
        "ece": ece
    }

In [12]:
def plot_comparison(results, save_dir="/content/drive/MyDrive/NCA/results"):
    os.makedirs(save_dir, exist_ok=True)

    metric_keys = ["overall_acc", "bal_acc", "mae", "brier", "nll", "entropy", "ece"]

    for metric in metric_keys:
        plt.figure(figsize=(8, 5))
        plt.plot(results["CNN"]["size"], results["CNN"][metric], marker='o', label="CNN")
        plt.plot(results["NCA"]["size"], results["NCA"][metric], marker='s', label="NCA")

        plt.title(f"{metric.replace('_', ' ').title()} vs Resolution")
        plt.xlabel("Image Size")
        plt.ylabel(metric.replace('_', ' ').title())
        plt.xticks(results["CNN"]["size"])  # saubere Ticks
        plt.legend()
        plt.grid(True)

        fname = os.path.join(save_dir, f"comparison_{metric}.png")
        plt.savefig(fname)
        plt.close()
        print(f"Saved: {fname}")

In [13]:
results = {
    "CNN": {"size": [], "overall_acc": [], "bal_acc": [], "mae": [], "brier": [], "nll": [], "entropy": [], "ece": []},
    "NCA": {"size": [], "overall_acc": [], "bal_acc": [], "mae": [], "brier": [], "nll": [], "entropy": [], "ece": []}
}

for size in [28, 64, 128, 224]:
    print(f"\n==============================")
    print(f"Resolution: {size}x{size}")
    loader = get_loader(size)

    print("CNN:")
    cnn_metrics = evaluate(cnn, loader, name="CNN", size=size)
    for k in cnn_metrics:
        results["CNN"][k].append(cnn_metrics[k])
    results["CNN"]["size"].append(size)

    print("NCA:")
    nca_metrics = evaluate(nca, loader, name="NCA", size=size, is_NCA=True)
    for k in nca_metrics:
        results["NCA"][k].append(nca_metrics[k])
    results["NCA"]["size"].append(size)


Resolution: 28x28
CNN:

CNN @ 28x28
Overall Accuracy: 0.8216
Balanced Accuracy: 0.7853
Mean Absolute Error (MAE): 0.5536

Uncertainty Quantification:
Brier Score: 0.2595
NLL (Cross-Entropy)  : 0.7132
Mean Predictive Entropy: 0.3278
Expected Calibration Error (ECE): 0.0574
              precision    recall  f1-score   support

           0     0.9337    0.9895    0.9608      1338
           1     0.8378    1.0000    0.9117       847
           2     0.3957    0.7552    0.5193       339
           3     0.8424    0.7839    0.8121       634
           4     0.9377    0.7420    0.8285      1035
           5     0.8349    0.7432    0.7864       592
           6     0.8912    0.6856    0.7750       741
           7     0.5175    0.5273    0.5224       421
           8     0.8871    0.8410    0.8634      1233

    accuracy                         0.8216      7180
   macro avg     0.7864    0.7853    0.7755      7180
weighted avg     0.8446    0.8216    0.8260      7180

Confusion matrix save

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))



NCA @ 224x224
Overall Accuracy: 0.2492
Balanced Accuracy: 0.3073
Mean Absolute Error (MAE): 2.3504

Uncertainty Quantification:
Brier Score: 1.1674
NLL (Cross-Entropy)  : 5.2557
Mean Predictive Entropy: 0.7428
Expected Calibration Error (ECE): 0.4782
              precision    recall  f1-score   support

           0     0.0000    0.0000    0.0000      1338
           1     0.2036    1.0000    0.3383       847
           2     0.1912    0.7788    0.3070       339
           3     0.0000    0.0000    0.0000       634
           4     0.0459    0.0048    0.0087      1035
           5     0.6343    0.3868    0.4806       592
           6     0.4063    0.5884    0.4807       741
           7     0.0000    0.0000    0.0000       421
           8     0.1356    0.0065    0.0124      1233

    accuracy                         0.2492      7180
   macro avg     0.1797    0.3073    0.1809      7180
weighted avg     0.1572    0.2492    0.1470      7180

Confusion matrix saved to: /content/drive/M

In [14]:
plot_comparison(results)

Saved: /content/drive/MyDrive/NCA/results/comparison_overall_acc.png
Saved: /content/drive/MyDrive/NCA/results/comparison_bal_acc.png
Saved: /content/drive/MyDrive/NCA/results/comparison_mae.png
Saved: /content/drive/MyDrive/NCA/results/comparison_brier.png
Saved: /content/drive/MyDrive/NCA/results/comparison_nll.png
Saved: /content/drive/MyDrive/NCA/results/comparison_entropy.png
Saved: /content/drive/MyDrive/NCA/results/comparison_ece.png
